In [1]:
from datasets import load_dataset
from dataset_helper import encode, to_tf_dataset
import os
import json
import datetime
import tensorflow as tf
from scheduler import CustomSchedule
from model import SnapthatT5
from dataset_helper import create_dataset

/Users/linz1/miniconda3/envs/tf_parallel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
print("load data")

train_dataset = load_dataset('squad', split='train[:10%]')
valid_dataset = load_dataset('squad', split='validation[:10%]')
print("load done")
print("dataset feature: ", train_dataset.features)

train_ds = train_dataset.map(encode)
valid_ds = valid_dataset.map(encode)
# ex = next(iter(train_ds))
# print("Example data from the mapped dataset: \n")
# for e in ex:
#     print(e, type(ex[e]), ex[e])
print("encode done")

tf_train_ds = to_tf_dataset(train_ds)
tf_valid_ds = to_tf_dataset(valid_ds)

ex = next(iter(tf_train_ds))
print("Example data from the tf dataset: \n")
for e in ex:
    print(e, type(ex[e]), ex[e])

print("to_tf_dataset done")

load data


load done
dataset feature:  {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}
encode done
ds:  <_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(250,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(250,), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(54,), dtype=tf.int64, name=None), 'decoder_attention_mask': TensorSpec(shape=(54,), dtype=tf.int64, name=None)}>
Example data from the dataset: 

input_ids tf.Tensor(
[ 1525   834   526    10   304  4068   410     8 16823  3790     3 18280
  2385    16   507  3449    16   301  1211  1395  1410    58  2625    10
 30797   120     6     8   496    65     3     9  6502  1848     5    71
  2916     8  5140  5450    31     7  2045 22161    19     3     9 

In [8]:
import numpy as np
tensorboard_logdir = "./data/experiments/t5/logs"
checkpoint_savedir = "./data/experiments/t5/models"

batch_size = 4
train_steps = int(np.ceil(len(train_dataset)/batch_size))
valid_steps = int(np.ceil(len(valid_dataset)/batch_size))

# os.environ["TF_CONFIG"] = json.dumps({
#     "cluster": {
#         "worker": ["host1:port", "host2:port"],
#         "ps": ["host4:port", "host5:port"]
#     },
#    "task": {"type": "worker", "index": 1}
# })

In [9]:
# tf_config = json.loads(os.environ["TF_CONFIG"])
# num_workers = len(tf_config["cluster"]["worker"])
# global_batch_size = num_workers * 4
global_batch_size = batch_size
# strategy = tf.distribute.MultiWorkerMirroredStrategy()
strategy = tf.distribute.MirroredStrategy()

tf_train_ds = create_dataset(tf_train_ds, batch_size=global_batch_size, shuffling=True)
tf_valid_ds = create_dataset(tf_valid_ds, batch_size=global_batch_size, shuffling=False)

tensorboard_logpath = tensorboard_logdir + "/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_logpath)

checkpoint_filepath = checkpoint_savedir + "/T5-{epoch:04d}-{val_loss:.4f}.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    mode='min',
    save_best_only=True,
)

# with strategy.scope():
callbacks = [tensorboard_callback, checkpoint_callback]
metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name='accuracy')]
# learning_rate = CustomSchedule()
learning_rate=1e-6
optimizer = tf.keras.optimizers.Adam(learning_rate)
model = SnapthatT5.from_pretrained("t5-small")
model.compile(optimizer=optimizer, metrics=metrics)

model.fit(
    tf_train_ds,
    epochs=1,
    steps_per_epoch=train_steps,
    validation_data=tf_valid_ds, 
    validation_steps=valid_steps,
    callbacks=callbacks, 
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
All PyTorch model weights were used when initializing SnapthatT5.

Some weights or buffers of the TF 2.0 model SnapthatT5 were not initialized from the PyTorch model and are newly initialized: ['total', 'count']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2190/2190 [==============================] - ETA: 0s - accuracy: 0.6316 - loss: 4.9978 - lr: 1.0000e-06

INFO:tensorflow:Assets written to: ./data/experiments/t5/models/T5-0001-1.2095.ckpt/assets


INFO:tensorflow:Assets written to: ./data/experiments/t5/models/T5-0001-1.2095.ckpt/assets


2190/2190 [==============================] - 854s 383ms/step - accuracy: 0.6316 - loss: 4.9978 - lr: 1.0000e-06 - val_accuracy: 0.9146 - val_loss: 1.2095


In [34]:
def gen():
    (x_train, y_train) = np.random.rand(1), np.random.rand(1) + 0.5
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    yield x_train, y_train

dataset = tf.data.Dataset.from_generator(
     gen,
     output_signature=(
         tf.TensorSpec(shape=(1), dtype=tf.float32),
         tf.TensorSpec(shape=(1), dtype=tf.int64)))

dataset = dataset.repeat().batch(4)

list(dataset.take(1))


[(<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
  array([[0.00049844],
         [0.00160483],
         [0.00249983],
         [0.00212634]], dtype=float32)>,
  <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
  array([[1],
         [0],
         [1],
         [1]])>)]

In [28]:
(x_train, y_train) = np.random.rand(10, 1), np.random.rand(10, 1) + 0.5
# The `x` arrays are in uint8 and have values in the [0, 255] range.
# You need to convert them to float32 with values in the [0, 1] range.
x_train = x_train / np.float32(255)
y_train = y_train.astype(np.int64)
dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(60000)
    .repeat()
    .batch(4)
)

list(dataset.take(1))

[(<tf.Tensor: shape=(4, 1), dtype=float64, numpy=
  array([[0.00113372],
         [0.00214739],
         [0.00045729],
         [0.0035719 ]])>,
  <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
  array([[0],
         [0],
         [0],
         [1]])>)]